# Chapter 19. Dynamic Attributes and Properties

Data attributes and methods are collectively known as attributes in PYthon: a method is just an attribute that is callable. Besides data attributes and methods, we can also create propertities, which can be used to replace a public data attribute wit accessor methods (i.e., getter/setter), without chaning the class interface.

Besides properties, Python provides a rich API for controlling attribute access and implementing dynamic attributes. The interpreter calls special methods such as `__getattr__` and `__setattr__` to evaluate attribute access using dot notation.

## Using a Property for Attrbute Validation

### LineItem Take #1: Class for an item in an Order

In [1]:
class LineItem:
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [2]:
raisins = LineItem('Golden rasins', 10, 6.95)
raisins.subtotal()

69.5

In [4]:
raisins.weight = -20  # garbage in ...
raisins.subtotal()    # garbage out ...

-139.0

    We found that customers could order a negative quantity of books! And we would credit their credit card with the price and, I assume, wait around for them to ship the books.
    
    - Jeff Bezos
    Founder and CEO of Amazon.com

### LineItem Take #2 A Validating Property

In [5]:
class LineItem:
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price
    
    @property
    def weight(self):
        # The actual value is stored in private attribute __weight
        return self.__weight
    
    # The decoarte getter has .setter attribute, which is also a 
    # decorator; this ties the getter and setter together.
    @weight.setter
    def weight(self, value):
        if value > 0:
            self.__weight = value
        else:
            raise ValueError('value must be > 0')

Now we have protected weight from users providing negative values.

## A Proper Look at Properties

Although often used as a decorator, the property build-in is actually a class.

    property(fget=None, fset=None, fdel=None, doc=None)

The property type was added in Python 2.2, but the @decorator syntax appeared only in Python 2.4.

property without using decorators:

In [6]:
class LineItem:
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.price * self.weight
    
    def get_weight(self):
        return self.__weight
    
    def set_weight(self, value):
        if value > 0:
            self.__weight = value
        else:
            raise ValueError('value must be > 0')
            
    weight = property(get_weight, set_weight)

### Properties Override Instance Attributes

Properties are always class attributes, but they actually manage attribute access  in the instance of the class.

Instance attributes shadows class data attribute

In [7]:
class Class:
    
    data = 'the class data attr'
    
    @property
    def prop(self):
        return 'the prop value'

In [9]:
obj = Class()
# vars return the __dict__ of obj, 
# showing it has no  instance attribute
vars(obj) 

{}

In [10]:
obj.data = 'bar'
vars(obj)

{'data': 'bar'}

In [11]:
Class.data

'the class data attr'

Instance attribute does not shadow class property

In [12]:
Class.prop

In [13]:
obj.prop

'the prop value'

In [17]:
# Trying to set an instance prop attribute fails.
obj.prop = 'foo'

AttributeError: can't set attribute

In [18]:
# Putting 'prop' directly in the object.__dict__ works.
obj.__dict__['prop'] = 'foo'
vars(obj)

{'data': 'bar', 'prop': 'foo'}

In [19]:
# Overwritting Class.prop destroys the property object.
Class.prop = 'baz'

In [21]:
obj.prop

'foo'

New class property shadows existing instance attribute.

In [22]:
obj.data

'bar'

In [23]:
Class.data

'the class data attr'

In [29]:
# Oerwrite Class.data with a new property
Class.data = property(lambda self: 'the "data" prop value')

In [30]:
# obj.data is now shadowed by the Class.data property
obj.data

'the "data" prop value'

In [31]:
del Class.data

In [32]:
obj.data

'bar'

**The main point of this section is that an expression like `obj.attr` does not search for attr starting with obj. The search actually starts at `obj.__class__`, and only if there is no property named attr in the class, Python looks in the object instance itself. This rule applies not only to properties but to a whole category of descriptors, the overridding descriptors.**

## Essential Attributes and Functions for Attribute Handling

### Special Attributes that Affect Attribute Handling

The behavior of many of the functions and special methods listed in the following sections depend on three special attributes:

`__class__`
    
    A reference to the object's class(i.ee, obj.__class__ is the same as type(obj)). Python looks for special methods such as __getattr__ only in an object's class, and not in the instances themselves.

`__dict__`

    A mapping that stores the writable attributes of an object or class. An object that has a __dict__ can have arbitrary new attributes set at any time. If a class has a __slots__ attributes, then its instances may not have a __dict__.

`__slots__`

    An attribute that may be defined in a class to limit the attributes its instance can have. __slots__ is a tuple of string naming the allowed attributes. IF the __dict__ name is not in __slots__, then the instances of that class will not have a __dict__ of their own, and only the named attributes will be allowed in them.

### Built-In Functions for Attribute Handling

These five built-in functions perform object attribute reading, writing, and introspection:

`dir([object])`
    
    List most attributes of the object. The official docs say dir is intended for interactive use so it does not provide a comprehensive list of attributes. The __dict__ attribute itself is not listed by dir, but the __dict__ keys are listed. Several special attributes of classes, such as __mro__, __bases__, and __name__ are not listed by dir either.
    
`getattr(object, name[, default])`

    Gets the attribute identified by the name string from the object. This may fetch an attribute from the object's class or from a superclass. If no such attribute exits, getattr raise AttributeError or returns the default value, if given.
    
`setattr(object, name, value)`

    Assigns the value to the named attribute of object, if the object allow it. This may create a new attribute or overwirte an exising one.
    
`vars([object])`

    Return the __dict__ of object; vars can't deal with instances of classes that define __slots__ and don't have a __dict__. Without an argument, the vars() does the same as locals(): return a dict representing the local scope.

### Special Methods for Attribute Handling

When implemented in a user-defined class, the speical methods listed here handle attribute retrivel, setting, deletion, and listing.

Attributes access using either dot notation or the built-in functions getattr, hasatr, and setattr trigger the appropriate special methods listed here. 

`__delattr__(self, name)`

    Always called when there is an attempt to delete an attribute using the del statement; e.g., del obj.attr triggers Class.__delattr__(obj, 'attr')
    
`__dir__(self)`

    Called when dir is invoked on the object, to provide a listing of attributes; e.g., dir(obj) triggers Class.__dir__(obj).
    
`__getattr__(self, name)`

    Called only when an attempt to retrieve the named attribute fails, after the obj, Class, and its superclass are searched.
    
`__getattribute__(self, name)`

    Always called when there is an attempt to retrieve the named attribute, except when the attribute sought is a special attribute or method. Dot notation and the getattr and hasattr built-ins trigger the method. __getattr__ is only invoked after __getatttribute__, and only when __getattribute__ raise AttributeError. To retrieve attributes of the instance obj  without triggering an infinite recursion, implementation of __getattribute__ should use super().__getattribute__(obj, name)
    
`__setattr__(self, name, value)`

    Always called when there is an attempt to set the named attribute. Dot notation and the setattr built-in trigger this method;  e.g., both obj.attr = 42 and setattr(obj, 'attr', 42) trigger Class.__setattr__(obj, 'attr', 42).